In [1]:
import pandas as pd
import numpy as np
import glob
import ntpath
from IPython.display import display, HTML

In [142]:
file_name =  "/home/jevjev/causal_inference_data/"
files = glob.glob(file_name+"*.xls")

In [145]:
def read_xls_source(file_name: str):
    dfs = dict()
    for sheet_name in ["Patients", "Samples", "Expression"]:
        df = pd.read_excel(io=file_name, sheet_name=sheet_name)
        dfs[sheet_name] = df
    return dfs

def get_counts(dfs, columns=["Pat_Adjuvant_Chemo", "Pat_Neoadjuvant_Chemo", "Pat_Radiation", "Pat_Died", "Pat_Recurrence"]):
    summary = dict()
    for c in columns:
        unique, count = np.unique(dfs['Patients'][c].dropna().astype(np.int32), return_counts=True)
        if len(unique)==0:
            count = [0,0]
        if len(unique)==1:
            _count = [0,0]
            _count[unique[0]] = count[0]
            count = _count
        summary[c] = count
    dfs["Summary"] = pd.DataFrame(summary)
    return dfs

In [146]:
dfs = dict()
for f in files:
    name = ntpath.basename(f).split("_")[1:3]
    name = '_'.join(name).split(".")[0]
    print(name)
    df = read_xls_source(f)
    df = get_counts(df)
    display(df['Summary'])
    dfs[name] = df

TCGA_LUSC


,Pat_Adjuvant_Chemo,Pat_Neoadjuvant_Chemo,Pat_Radiation,Pat_Died,Pat_Recurrence
0,103,494,139,343,0
1,51,7,15,158,0


Der_2014


,Pat_Adjuvant_Chemo,Pat_Neoadjuvant_Chemo,Pat_Radiation,Pat_Died,Pat_Recurrence
0,181,0,181,106,126
1,0,0,0,75,51


Botling_2013


...

,Pat_Adjuvant_Chemo,Pat_Neoadjuvant_Chemo,Pat_Radiation,Pat_Died,Pat_Recurrence
0,48,48,48,25,22
1,0,0,0,23,26


TCGA_LUAD


,Pat_Adjuvant_Chemo,Pat_Neoadjuvant_Chemo,Pat_Radiation,Pat_Died,Pat_Recurrence
0,117,512,142,390,0
1,37,3,13,126,0


Okayama_2012


,Pat_Adjuvant_Chemo,Pat_Neoadjuvant_Chemo,Pat_Radiation,Pat_Died,Pat_Recurrence
0,204,0,204,174,150
1,0,0,0,30,54


Select datasets for which `Pat_Adjuvant_Chemo` has `1`

In [147]:
cumulative = [0, 0]
for name, df in list(dfs.items()):
    if df['Summary'].Pat_Adjuvant_Chemo.iloc[1] != 0:
        cumulative[0] += df['Summary'].Pat_Adjuvant_Chemo.iloc[0]
        cumulative[1] += df['Summary'].Pat_Adjuvant_Chemo.iloc[1]
    else:
        del dfs[name]

In [148]:
dfs.keys()

dict_keys(['TCGA_LUSC', 'Botling_2013', 'Jones_2004', 'Shedden_2008', 'TCGA_LUAD'])

Both TCGA datasets appear to not have recurrance data, and therefore have to be dropped 

In [149]:
cumulative = [0, 0]
for name, df in list(dfs.items()):
    if df['Summary'].Pat_Adjuvant_Chemo.iloc[1] != 0:
        cumulative[0] += df['Summary'].Pat_Adjuvant_Chemo.iloc[0]
        cumulative[1] += df['Summary'].Pat_Adjuvant_Chemo.iloc[1]

In [150]:
cumulative

[534, 221]

Iterate over datasets and combine them into a single dataset

In [266]:
combined = []
for name, df in dfs.items():
    patients = df['Patients']
    samples = df['Samples']
    expression = df['Expression']
    patients = patients.dropna(subset=["Pat_Adjuvant_Chemo", 
                                       "Pat_Overall_Survival_Months", 
                                       "Pat_Died"], how='all', thresh=3).set_index("Pat_ID")
    pat_id = patients.index
    samples = samples.set_index('Sam_Patient').loc[pat_id]["Sam_Name"].values
    patients['Sam_Name'] = samples
    patients = patients.set_index('Sam_Name')
    available = expression.columns.values.tolist()
    u = list(set.intersection(set(available), set(samples.tolist())))
    expression = expression[u].transpose()
    patients = patients.loc[u].join(expression)
    patients = patients.loc[:, patients.isnull().mean() < 0.3]
#     patients.to_csv("{}.csv".format(name))
    combined.append(patients)
    print(name+": {} patients with chemo, survival and status and genetic features available".format(len(u)))

Botling_2013: 100 patients with chemo, survival and status and genetic features available
Jones_2004: 25 patients with chemo, survival and status and genetic features available
Shedden_2008: 69 patients with chemo, survival and status and genetic features available


In [267]:
combined = pd.concat(combined)
combined = combined.loc[:, combined.isnull().mean() < 0.3]

In [276]:
combined.to_csv("combined.csv")